In [4]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import pickle
from utils import *

In [5]:
root = '/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/cortices/'

R = Retina(gpu=False)
R.loadLoc('/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/ret50k_loc.pkl')
R.loadCoeff('/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/ret50k_coeff.pkl')
fixation = (1080/2,1920/2)
shape = (1080, 1920)
R.prepare(shape, fixation)

C = Cortex(gpu=False)
C.loadLocs(root+'50k_Lloc_tight.pkl', root+'50k_Rloc_tight.pkl')
C.loadCoeffs(root+'50k_Lcoeff_tight.pkl', root+'50k_Rcoeff_tight.pkl')

In [ ]:
import zmq
context = zmq.Context()

print("Connecting to Client…")
client_socket = context.socket(zmq.PULL)
client_socket.bind("tcp://*:5555")

ret = False
cort = False
while True:
    data = client_socket.recv()
    print(data)
    if data == b'Ret':
        ret = True
        continue
    if data == b'Cort':
        cort = True
        continue
    
    if data == b'Done':
        cv2.destroyWindow('Image')
        cv2.waitKey(1)
        ret = False
        cort = False
        continue
    
    V = pickle.loads(data)
    if ret:
        image = R.backproject_tight(V, (1080,1920,3), fixation)
    elif cort:
        image = C.cort_img(V)
    else:
        image = V      
    
    cv2.imshow('Image', image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


In [ ]:
video_path = '/Users/nicholassperry/Desktop/Dataset.nosync/Dataset/Train/Gargoyle.MOV'
vidcap = cv2.VideoCapture(video_path)
success, img = vidcap.read()

count = 1
while success:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if count ==1:
        print(img.shape)
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    success, img = vidcap.read()
    count += 1

cv2.destroyWindow('Image')
cv2.waitKey(1)
vidcap.release()